En esta instalación es muy importante que no reinicies cuando te salte un cuadro de diálogo solicitándolo

In [ ]:
#git clone https://github.com/OmicsML/CellPLM.git
%cd CellPLM
#pip install .
#pip install gdown
#pip install datasets
#pip install scanpy anndata

Cloning into 'CellPLM'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 156 (delta 59), reused 117 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (156/156), 359.98 KiB | 1.33 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/CellPLM
Processing /content/CellPLM
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.6/270.6 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.9/960.9 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.6 MB/s eta 0:00

In [2]:
import hdf5plugin
import anndata as ad
import scanpy as sc
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from CellPLM.utils import set_seed
from CellPLM.pipeline.cell_embedding import CellEmbeddingPipeline
from CellPLM.pipeline.cell_type_annotation import CellTypeAnnotationPipeline, CellTypeAnnotationDefaultPipelineConfig, CellTypeAnnotationDefaultModelConfig
import gdown
import os
from datasets import load_dataset
import numpy as np
import torch
from sklearn.model_selection import train_test_split


In [3]:
set_seed(42)

Bajamos los parámetros del modelo pre-entrenado y los guardamos en la carpeta ckpt

In [4]:
output_dir = "/content/CellPLM/ckpt/20230926_85M.best.ckpt"
file_id = "1IEoo_xPIpqnJluty2nQQVolb534KaWW3"

gdrive_url = f"https://drive.google.com/uc?id={file_id}"
gdown.download(gdrive_url, output=output_dir, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1IEoo_xPIpqnJluty2nQQVolb534KaWW3
From (redirected): https://drive.google.com/uc?id=1IEoo_xPIpqnJluty2nQQVolb534KaWW3&confirm=t&uuid=f909ce05-dea7-4c4d-b68c-9ae39d29a4ca
To: /content/CellPLM/ckpt/20230926_85M.best.ckpt
100%|██████████| 907M/907M [00:12<00:00, 70.2MB/s]


'/content/CellPLM/ckpt/20230926_85M.best.ckpt'

# DATOS HELICAL

In [ ]:
file_id = "19PYEpPeFm-YFlgD1459ms9j-TDhDja_W"

url = f"https://drive.google.com/uc?id={file_id}"

output_path = "/content/CellPLM/data"
gdown.download(url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=19PYEpPeFm-YFlgD1459ms9j-TDhDja_W
From (redirected): https://drive.google.com/uc?id=19PYEpPeFm-YFlgD1459ms9j-TDhDja_W&confirm=t&uuid=6aded640-e619-433a-9930-eff983497f21
To: /content/CellPLM/data
100%|██████████| 553M/553M [00:10<00:00, 51.2MB/s]


'/content/CellPLM/data'

In [ ]:
ds = sc.read_h5ad("/content/CellPLM/data/data3ntxi0h8.part")


sc.pp.highly_variable_genes(ds, n_top_genes=3000, flavor='seurat_v3', inplace=True)
ds = ds[:, ds.var["highly_variable"]]


train1_ids, test_ids = train_test_split(
    np.arange(ds.n_obs), test_size=0.2, random_state=42
)

train2_ids, valid_ids = train_test_split(
    train1_ids, test_size=0.2, random_state=42
)

train_dataset = ds[train2_ids].copy()
valid_dataset = ds[valid_ids].copy()
test_dataset = ds[test_ids].copy()

sc.pp.normalize_total(train_dataset)
sc.pp.log1p(train_dataset)

sc.pp.normalize_total(valid_dataset)
sc.pp.log1p(valid_dataset)

sc.pp.normalize_total(test_dataset)
sc.pp.log1p(test_dataset)

cell_types_train = list(np.array(train_dataset.obs["LVL2"].tolist())) # Cambiar niveles de clasificación
cell_types_test = list(np.array(test_dataset.obs["LVL2"].tolist()))

label_set = set(cell_types_train) | set(cell_types_test)
class_id_dict = dict(zip(label_set, [i for i in range(len(label_set))]))
id_class_dict = {v: k for k, v in class_id_dict.items()}

for i in range(len(cell_types_train)):
    cell_types_train[i] = class_id_dict[cell_types_train[i]]

for i in range(len(cell_types_test)):
    cell_types_test[i] = class_id_dict[cell_types_test[i]]

y_true_test = test_dataset.obs["LVL2"].astype('category').cat.codes.to_numpy() # Modificar

adata_train_copy = train_dataset.copy()
valid_train_copy = valid_dataset.copy()
adata_test_copy = test_dataset.copy()

adata_train_copy.obs['split'] = 'train'
valid_train_copy.obs['split'] = 'valid'
adata_test_copy.obs['split'] = 'test'

from anndata import concat
data_helical = concat([adata_train_copy, valid_train_copy, adata_test_copy],
                       join='outer',
                       index_unique=None)

# DATOS ESCLEROSIS MÚLTIPLE

In [5]:
output_dir = "../data/ms"
#output_dir = "/kaggle/working/data/ms"

if os.path.exists(output_dir):

    print("data already downloaded!!")

else:

    folder_id = "1Qd42YNabzyr2pWt9xoY4cVMTAxsNBt4v"
    gdown.download_folder(f"https://drive.google.com/drive/folders/{folder_id}", output=output_dir, quiet=False)

Retrieving folder contents


Processing file 1Vp51xYAOEu5_Wd7lilnatvMJ7tr2bsbl .DS_Store
Processing file 1bV1SHKVZgkcL-RmmuN51_IIUJTSJbXOi c_data.h5ad
Processing file 1casFhq4InuBNhJLMnGebzkRXM2UTTeQG filtered_ms_adata.h5ad


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1Vp51xYAOEu5_Wd7lilnatvMJ7tr2bsbl
To: /content/data/ms/.DS_Store
100%|██████████| 6.15k/6.15k [00:00<00:00, 15.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1bV1SHKVZgkcL-RmmuN51_IIUJTSJbXOi
To: /content/data/ms/c_data.h5ad
100%|██████████| 20.6M/20.6M [00:00<00:00, 45.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1casFhq4InuBNhJLMnGebzkRXM2UTTeQG
To: /content/data/ms/filtered_ms_adata.h5ad
100%|██████████| 47.1M/47.1M [00:00<00:00, 53.2MB/s]
Download completed


In [6]:
data_control = ad.read_h5ad(f'../data/ms/c_data.h5ad')
data_enfermos = ad.read_h5ad(f'../data/ms/filtered_ms_adata.h5ad')

train_indices, valid_indices = train_test_split(
    np.arange(data_control.n_obs), test_size=0.2, random_state=42
)

adata_train_copy = data_control[train_indices].copy()
valid_train_copy = data_control[valid_indices].copy()


adata_train_copy.obs['split'] = 'train'
valid_train_copy.obs['split'] = 'valid'
data_enfermos.obs['split'] = 'test'

data_enfermos.var = data_enfermos.var.set_index('gene_name')
data_enfermos.var["gene_name"] = data_enfermos.var.index

from anndata import concat
data_ms = concat([adata_train_copy, valid_train_copy, data_enfermos],
                       join='inner',
                       index_unique=None)

y_true_test = data_enfermos.obs["celltype"].astype('category').cat.codes.to_numpy()

cell_types_train = list(np.array(adata_train_copy.obs["celltype"].tolist()))
cell_types_test = list(np.array(data_enfermos.obs["celltype"].tolist()))

label_set = set(cell_types_train) | set(cell_types_test) # Unimos ambos conjuntos
class_id_dict = dict(zip(label_set, [i for i in range(len(label_set))]))
id_class_dict = {v: k for k, v in class_id_dict.items()}


#Sustituimos los enteros por sus identificadores numéricos
for i in range(len(cell_types_train)):
    cell_types_train[i] = class_id_dict[cell_types_train[i]]

for i in range(len(cell_types_test)):
    cell_types_test[i] = class_id_dict[cell_types_test[i]]


/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:740: UserWarning: 
AnnData expects .var.index to contain strings, but got values like:
    ['CFH', 'BAD', 'LAP3', 'CD99', 'WNT16']

    Inferred to be: categorical

  value_idx = self._prep_dim_index(value.index, attr)


In [ ]:
data_ms

AnnData object with n_obs × n_vars = 21312 × 3000
    obs: 'Sample Characteristic[organism]', 'Sample Characteristic Ontology Term[organism]', 'Sample Characteristic[individual]', 'Sample Characteristic Ontology Term[individual]', 'Sample Characteristic[sex]', 'Sample Characteristic Ontology Term[sex]', 'Sample Characteristic[age]', 'Sample Characteristic Ontology Term[age]', 'Sample Characteristic[developmental stage]', 'Sample Characteristic Ontology Term[developmental stage]', 'Sample Characteristic[organism part]', 'Sample Characteristic Ontology Term[organism part]', 'Sample Characteristic[sampling site]', 'Sample Characteristic Ontology Term[sampling site]', 'Sample Characteristic[disease]', 'Sample Characteristic Ontology Term[disease]', 'Sample Characteristic[organism status]', 'Sample Characteristic Ontology Term[organism status]', 'Sample Characteristic[cause of death]', 'Sample Characteristic Ontology Term[cause of death]', 'Sample Characteristic[clinical history]', 'Sample 

In [ ]:
data_ms.obs['split'].value_counts()

,count
split,
test,13468
train,6275
valid,1569


# Modelo CellPLM
## Aplicado a esclerosis múltiple

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
PRETRAIN_VERSION = '20230926_85M'
DEVICE = device

In [8]:
pipeline_config = CellTypeAnnotationDefaultPipelineConfig.copy()
num_workers =  min(len(os.sched_getaffinity(0)), 16)
pipeline_config['workers'] = num_workers # truco del código fuente de scGPT
pipeline_config['epochs'] = 1200 # Probar con distintas, me parece que esta 1 no es equivalente a la del resto de modelos
pipeline_config['lr'] = 1e-4 # creo que es altamente sensible



model_config = CellTypeAnnotationDefaultModelConfig.copy()
model_config['out_dim'] = data_ms.obs['celltype'].nunique()
print(pipeline_config)
model_config

{'es': 200, 'lr': 0.0001, 'wd': 1e-07, 'scheduler': 'plat', 'epochs': 1200, 'max_eval_batch_size': 100000, 'hvg': 3000, 'patience': 25, 'workers': 2}


{'drop_node_rate': 0.3,
 'dec_layers': 1,
 'model_dropout': 0.5,
 'mask_node_rate': 0.75,
 'mask_feature_rate': 0.25,
 'dec_mod': 'mlp',
 'latent_mod': 'ae',
 'head_type': 'annotation',
 'max_batch_size': 70000,
 'out_dim': 18}

In [9]:
pipeline = CellTypeAnnotationPipeline(pretrain_prefix=PRETRAIN_VERSION, # Specify the pretrain checkpoint to load
                                      overwrite_config=model_config,  # This is for overwriting part of the pretrain config
                                      pretrain_directory='/content/CellPLM/ckpt')
pipeline.model

OmicsFormer(
  (embedder): OmicsEmbeddingLayer(
    (act): ReLU()
    (norm0): GroupNorm(4, 1024, eps=1e-05, affine=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (extra_linear): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GroupNorm(4, 1024, eps=1e-05, affine=True)
    )
    (pe_enc): Sinusoidal2dPE(
      (pe_enc): Embedding(10000, 1024)
    )
    (feat_enc): OmicsEmbedder()
  )
  (mask_model): MaskBuilder()
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x FlowformerLayer(
        (self_attn): Flow_Attention(
          (query_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (key_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (value_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (out_projection): Linear(in_features=1024, out_features=1024, bias=True)
         

In [10]:
pipeline.fit(data_ms, # An AnnData object
            pipeline_config, # The config dictionary we created previously, optional
            split_field = 'split', #  Specify a column in .obs that contains split information
            train_split = 'train',
            valid_split = 'valid',
            label_fields = ['celltype'],
            device = device) # Specify a column in .obs that contains cell type labels

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/pipeline/__init__.py:70: UserWarning: HVG number is larger than number of valid genes.
  warnings.warn('HVG number is larger than number of valid genes.')
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(
/content/CellPLM/CellPLM/utils/data.py:20: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(x.indptr, x.indices, x.data, (x.shape[0], x.shape[1])).to_sparse().float().c

After filtering, 2535 genes remain.


  0%|          | 0/1200 [00:00<?, ?it/s]/content/CellPLM/CellPLM/utils/sparse.py:37: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:644.)
  masked_x_seq = torch.sparse.FloatTensor(x.indices(),
  0%|          | 1/1200 [00:04<1:38:02,  4.91s/it]

...

Epoch 1197 | Train loss: 0.3225 | Valid loss: 0.7379
Train ACC: 0.8026 | Valid ACC: 0.7680 | Train f1: 0.8153 | Valid f1: 0.7938 | Train pre: 0.8579 | Valid pre: 0.8967


100%|█████████▉| 1199/1200 [1:04:23<00:03,  3.33s/it]

Epoch 1198 | Train loss: 0.3390 | Valid loss: 0.7379
Train ACC: 0.7854 | Valid ACC: 0.7673 | Train f1: 0.7959 | Valid f1: 0.7932 | Train pre: 0.8336 | Valid pre: 0.8962


100%|██████████| 1200/1200 [1:04:26<00:00,  3.22s/it]

Epoch 1199 | Train loss: 0.3181 | Valid loss: 0.7329
Train ACC: 0.8087 | Valid ACC: 0.7685 | Train f1: 0.8134 | Valid f1: 0.7943 | Train pre: 0.8478 | Valid pre: 0.8967


In [11]:
pipeline.predict(
                data_ms, # An AnnData object
                pipeline_config,# The config dictionary we created previously, optional
                device = device
            )

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(


After filtering, 2535 genes remain.


tensor([5, 3, 5,  ..., 3, 8, 6], device='cuda:0')

In [12]:
pipeline.score(data_ms, # An AnnData object
                pipeline_config, # The config dictionary we created previously, optional
                split_field = 'split', # Specify a column in .obs to specify train and valid split, optional
                target_split = 'test', # Specify a target split to predict, optional
                label_fields = ['celltype'],
                device = device
              )

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(


After filtering, 2535 genes remain.


{'acc': 0.7409325242042542,
 'f1_score': 0.7153164148330688,
 'precision': 0.7506636381149292,
 'recall': 0.7409325242042542}

# Modelo CellPLM
## Aplicado a Helical

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
PRETRAIN_VERSION = '20230926_85M'

In [ ]:
pipeline_config = CellTypeAnnotationDefaultPipelineConfig.copy()
num_workers =  min(len(os.sched_getaffinity(0)), 16)
pipeline_config['workers'] = num_workers # truco del código fuente de scGPT
pipeline_config['epochs'] = 1200 # Probar con distintas, me parece que esta 1 no es equivalente a la del resto de modelos
pipeline_config['lr'] = 1e-4 # creo que es altamente sensible



model_config = CellTypeAnnotationDefaultModelConfig.copy()
model_config['out_dim'] = data_helical.obs['LVL2'].nunique() # Cambiar niveles de predicción
print(pipeline_config)
model_config

{'es': 200, 'lr': 0.0001, 'wd': 1e-07, 'scheduler': 'plat', 'epochs': 1200, 'max_eval_batch_size': 100000, 'hvg': 3000, 'patience': 25, 'workers': 2}


{'drop_node_rate': 0.3,
 'dec_layers': 1,
 'model_dropout': 0.5,
 'mask_node_rate': 0.75,
 'mask_feature_rate': 0.25,
 'dec_mod': 'mlp',
 'latent_mod': 'ae',
 'head_type': 'annotation',
 'max_batch_size': 70000,
 'out_dim': 28}

In [ ]:
pipeline = CellTypeAnnotationPipeline(pretrain_prefix=PRETRAIN_VERSION, # Specify the pretrain checkpoint to load
                                      overwrite_config=model_config,  # This is for overwriting part of the pretrain config
                                      pretrain_directory='/content/CellPLM/ckpt')
pipeline.model

OmicsFormer(
  (embedder): OmicsEmbeddingLayer(
    (act): ReLU()
    (norm0): GroupNorm(4, 1024, eps=1e-05, affine=True)
    (dropout): Dropout(p=0.5, inplace=False)
    (extra_linear): Sequential(
      (0): Linear(in_features=1024, out_features=1024, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.5, inplace=False)
      (3): GroupNorm(4, 1024, eps=1e-05, affine=True)
    )
    (pe_enc): Sinusoidal2dPE(
      (pe_enc): Embedding(10000, 1024)
    )
    (feat_enc): OmicsEmbedder()
  )
  (mask_model): MaskBuilder()
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-3): 4 x FlowformerLayer(
        (self_attn): Flow_Attention(
          (query_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (key_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (value_projection): Linear(in_features=1024, out_features=1024, bias=True)
          (out_projection): Linear(in_features=1024, out_features=1024, bias=True)
         

In [ ]:
pipeline.fit(data_helical, # An AnnData object
            pipeline_config, # The config dictionary we created previously, optional
            split_field = 'split',
            train_split = 'train',
            valid_split = 'valid',
            label_fields = ['LVL2'], # Modificar
            device = device)

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/pipeline/__init__.py:70: UserWarning: HVG number is larger than number of valid genes.
  warnings.warn('HVG number is larger than number of valid genes.')
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(
/content/CellPLM/CellPLM/utils/data.py:20: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /pytorch/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(x.indptr, x.indices, x.data, (x.shape[0], x.shape[1])).to_sparse().float().c

After filtering, 2484 genes remain.


  0%|          | 0/1200 [00:00<?, ?it/s]/content/CellPLM/CellPLM/utils/sparse.py:37: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:644.)
  masked_x_seq = torch.sparse.FloatTensor(x.indices(),
  0%|          | 1/1200 [00:05<1:54:20,  5.72s/it]

...

Epoch 1197 | Train loss: 0.1227 | Valid loss: 1.4420
Train ACC: 0.5193 | Valid ACC: 0.4893 | Train f1: 0.5166 | Valid f1: 0.4938 | Train pre: 0.5283 | Valid pre: 0.5417


100%|█████████▉| 1199/1200 [1:27:25<00:04,  4.45s/it]

Epoch 1198 | Train loss: 0.1185 | Valid loss: 1.4273
Train ACC: 0.5476 | Valid ACC: 0.4902 | Train f1: 0.5405 | Valid f1: 0.4946 | Train pre: 0.5457 | Valid pre: 0.5420


100%|██████████| 1200/1200 [1:27:30<00:00,  4.38s/it]

Epoch 1199 | Train loss: 0.1194 | Valid loss: 1.4094
Train ACC: 0.4829 | Valid ACC: 0.4909 | Train f1: 0.4782 | Valid f1: 0.4955 | Train pre: 0.4786 | Valid pre: 0.5423


In [ ]:
pipeline.predict(
                data_helical, # An AnnData object
                pipeline_config,# The config dictionary we created previously, optional
                device = device
                )

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(


After filtering, 2484 genes remain.


tensor([ 7, 16, 15,  ...,  4, 12,  4], device='cuda:0')

In [ ]:
pipeline.score(data_helical, # An AnnData object
                pipeline_config, # The config dictionary we created previously, optional
                split_field = 'split',
                target_split = 'test',
                label_fields = ['LVL2'], # Modificar
                device = device
              )

INFO:biothings.client:querying 1-1000 ...


Automatically converting gene symbols to ensembl ids...


INFO:biothings.client:querying 1001-2000 ...
INFO:biothings.client:querying 2001-3000 ...
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
/content/CellPLM/CellPLM/utils/data.py:81: UserWarning: Batch labels not found in AnnData.obs. All cells are considered from the same sample by default.
  warnings.warn(


After filtering, 2484 genes remain.


{'acc': 0.4421578049659729,
 'f1_score': 0.4395614266395569,
 'precision': 0.4732854962348938,
 'recall': 0.4421578049659729}